# Nordlys RESTful API

This code retrieves the documents from DBPedia using the Nordlys RESTful API.

In [2]:
with open('query_list.txt', 'r') as f:
    text = f.read()
print(text)

INEX_LD-20120111	vietnam war movie
INEX_LD-20120112	vietnam war facts
INEX_LD-20120121	vietnam food recipes
INEX_LD-20120122	vietnamese food blog
INEX_LD-20120131	vietnam travel national park
INEX_LD-20120132	vietnam travel airports
INEX_LD-20120211	guitar chord tuning
INEX_LD-20120212	guitar chord minor
INEX_LD-20120221	guitar classical flamenco
INEX_LD-20120222	guitar classical bach
INEX_LD-20120231	guitar origin Russia
INEX_LD-20120232	guitar origin blues
INEX_LD-20120311	tango culture movies
INEX_LD-20120312	tango culture countries
INEX_LD-20120321	tango music  composers
INEX_LD-20120322	tango music instruments
INEX_LD-20120331	tango dance styles
INEX_LD-20120332	tango dance history
INEX_LD-20120411	bicycle sport races
INEX_LD-20120412	bicycle sport disciplines
INEX_LD-20120421	bicycle holiday towns
INEX_LD-20120422	bicycle holiday nature
INEX_LD-20120431	bicycle benefits health
INEX_LD-20120432	bicycle benefits environment
INEX_LD-20120511	female rock singers
INEX_LD-20120512	sout